#### Import Libraries

#### Read in clean data

#### Survive Vs. Make Visualization

## Fraction Fatal & Unijured vs Make: Substantial Damage

## Fraction Uninjured & Fatal vs Make: Destroyed Damage

#### Broad Phase of Flight vs Fraction Fatal

#### Broad Phase of Flight vs Fraction Unijured ?

#### Month vs Fraction Fatal

#### Year vs Fraction Fatal